In [1]:
!pip install torchtext==0.4.0

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import torchtext
import numpy as np

# 0.1 라이브러리 임포트

In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as toransforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

# 0.2 하이퍼파라미터 셋팅

In [4]:
# Hyper-prameters
batch_size = 128
num_epochs = 10

word_vec_size = 256
dropout_p = 0.3

hidden_size = 512
num_layers = 4

## yhk 추가
leaning_rate = 0.001 #디폴트 0.001

In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. SMS train, test dataset 가져오기

In [6]:
from data_loader import DataLoader

In [7]:
loaders = DataLoader(
    train_fn ='./sms.maxlen.uniq.shuf.train.tsv',
    batch_size = batch_size,
    valid_ratio =.2,
    device = -1,
    max_vocab = 999999,
    min_freq=5,
)

In [8]:
test_loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.test.tsv',
    batch_size = batch_size,
    valid_ratio = .01, #모두 train
    device = -1,
    max_vocab = 999999,
    min_freq = 5,
)

# 2. 대략적인 데이터 형태

In [9]:
print("|train| = ", len(loaders.train_loader.dataset),
     '|valid| = ', len(loaders.valid_loader.dataset))

vocab_size = len(loaders.text.vocab)
num_classes = len(loaders.label.vocab)
print('|vocab| = ', vocab_size, '|classes| = ', num_classes)

('|train| = ', 3722, '|valid| = ', 930)
('|vocab| = ', 1537, '|classes| = ', 2)


# 3. 데이터 로드함수
## 학습시킬 때 batch_size 단위로 끊어서 로드하기 위함

In [10]:
n = 3 # 샘플로 출력할 데이터 개수
for i, data in enumerate(loaders.train_loader):
    labels = data.label
    texts = data.text
    
    if i > n:
        break
    print("[%d]" %i)
    print("한 번에 로드되는 데이터 크기:", len(labels))
    
    #출력
    for j in range(n):
        label = labels[j].numpy()  # tensor -> numpy로 변환
        text = texts[j].numpy()
        print("label: ", label)
        print("text: ", text.shape)

[0]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (22,))
('label: ', array(1))
('text: ', (22,))
('label: ', array(1))
('text: ', (22,))
[1]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
[2]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (8,))
('label: ', array(0))
('text: ', (8,))
('label: ', array(0))
('text: ', (8,))
[3]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:',

# 4. 모델 선언 (RNN+SMS구현)

In [11]:
#Recurrent neural network(many-to-one)
class RNN(nn.Module):
    def __init__(self,
                input_size,    # vocab_size
                word_vec_size, # word embbeding vector 차원
                hidden_size,   # bidirectional LSTM 의 hedden state & cell state의 size 
                n_classes,
                num_layers=4,  # 쌓을 레이어 개수
                dropout_p=0.3
                ):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.num_layers = num_layers
        self.dropout_p = dropout_p
        
        #입력 차원(vocab_size), 출력 차원(word_vec_size)
        self.emb = nn.Embedding(input_size, word_vec_size) # 임베딩레이어 시작
        
        self.lstm = nn.LSTM(input_size = word_vec_size,
                           hidden_size = hidden_size,
                           num_layers = num_layers,
                           dropout = dropout_p,
                           batch_first = True,
                           bidirectional = True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        # LogSoftmax + NLLoss instead of Softmax + CrossEntropy
        self.activation = nn.LogSoftmax(dim = -1)
    
    def forward(self, x):
        x = self.emb(x)
        
        x,_ = self.lstm(x)
        
        out = self.activaion(self.fc(x[:,-1]))
        
        return out

In [12]:
model = RNN(input_size = vocab_size,
           word_vec_size = word_vec_size,
           hidden_size = hidden_size,
           n_classes = num_classes,
           num_layers = num_layers,
           dropout_p = dropout_p)

In [13]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    
    model.eval()
    for i, data in enumerate(dloader):
        texts = data.text.to(device)
        labels = data.label.to(device)
        
        output = model(texts)
        _,output_index = torch.max(output, 1)
        
        total += labels.size(0)
        correct += (output_index==labels).sum().float()
        
    model.train()
    return(100*correct/total).numpy()

In [14]:
print("Accuracy of Test Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

AttributeError: 'RNN' object has no attribute 'activaion'

# 5. loss, optimizer

In [15]:
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

NameError: name 'learning_rate' is not defined

# 6. 학습

In [32]:
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loaders.train_loader):
        texts = data.text.to(device)
        labels = data.label.to(device)
        
        print("[%d]" %i)
        
        outputs = model(texts)
        loss = loss_func(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1) % 10 ==0:
            print('Epoch[{}/{}], Step[{}/{}], Loss: {:.4f}, Accr: {:.2f}'
                 .format(epoch+1, num_epochs, i+1, total_step,
                        loss.item(),
                        ComputeAccr(loadrs.valid_loader, model)))

[0]


AttributeError: 'RNN' object has no attribute 'activaion'

# 7. Test

In [33]:
print("Accracy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

AttributeError: 'RNN' object has no attribute 'activaion'

In [ ]:
netname = './nets/rnn_weight.pkl'
torch.save(model, netname, )